<a href="https://colab.research.google.com/github/kimim9900/SImilar-materials-for-somescreen/blob/main/first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade git+https://github.com/qzhu2017/PyXtal.git@master


  Cloning https://github.com/qzhu2017/PyXtal.git (to revision master) to /tmp/pip-req-build-s3uwuq01
  Running command git clone --filter=blob:none --quiet https://github.com/qzhu2017/PyXtal.git /tmp/pip-req-build-s3uwuq01
  Resolved https://github.com/qzhu2017/PyXtal.git to commit e7a1b95d4a00e92950d9b8666c3626d09f3604c2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 69.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 k

In [2]:
!pip install rdkit-pypi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 17.2 MB/s eta 0:00:00


In [ ]:
# I download the file for Organic-1.db, 2, 3, and 4
# and I also downloaded mech.db

In [ ]:
import os
import sqlite3
import pandas as pd
from pyxtal.db import database

def check_table_exists(db_conn, table_name):
    cursr = db_conn.cursor()
    str = "select name from sqlite_master where type='table'"
    table_names = cursr.execute(str)
    print("Tables in the database:")
    tables=[]
    for name in table_names:
        tables.append(name[0])
    if table_name in tables:
        return True
    else:
        return False

# 1) Compute the template descriptor
mech_db = database('mech.db')
template_code = 'DAHMUX'
template_xtal = mech_db.get_pyxtal(template_code)
template_sph = template_xtal.get_spherical_images()

# Paths to the Organic database files
path_to_dbs = ['Organic-1.db', 'Organic-2.db', 'Organic-3.db', 'Organic-4.db']

results = []

# Loop over all database files
for path_to_db in path_to_dbs:

    # Create a connection to the Organic db
    conn = sqlite3.connect(path_to_db)

    if not check_table_exists(conn, 'systems'):
        print(f"'systems' table does not exist in the database {path_to_db}.")
        continue

    # Query to get all the unique_ids from 'systems' table in the current Organic db
    organic_codes_query = 'SELECT unique_id FROM systems'
    organic_codes = pd.read_sql_query(organic_codes_query, conn)['unique_id'].tolist()

    # Close the connection to the Organic db
    conn.close()

    # For each unique_id in organic db
    for unique_id in organic_codes:
        # Get the crystal structure and compute its descriptor
        organic_db = database(path_to_db)

        try:
            print(f"Details for unique_id {unique_id}:")
            xtal = organic_db.get_pyxtal(unique_id)
            print(xtal)
        except Exception as e:
            print(f"Failed to get details for unique_id {unique_id} due to error: {str(e)}")

        try:
            sph = xtal.get_spherical_images()
        except Exception as e:
            print(f"Failed to get spherical images for unique_id {unique_id} due to error: {str(e)}")
            continue

        # Compute the similarity (S) between sph1 and sph2
        try:
            S = template_sph.get_similarity(sph, M=10, cutoff=0.95).mean()
        except Exception as e:
            print(f"Failed to compute similarity for unique_id {unique_id} due to error: {str(e)}")
            continue

        # Save the information about (unique_id, S)
        results.append((unique_id, S))

# Export the (unique_id, S) to the csv file.
df = pd.DataFrame(results, columns=['unique_id', 'similarity'])
df.to_csv('similarity.csv', index=False)


Tables in the database:
Details for unique_id 1021e07653b9660d014c758778ebd19e:
Failed to get details for unique_id 1021e07653b9660d014c758778ebd19e due to error: cannot find the entry from 1021e07653b9660d014c758778ebd19e
Failed to get spherical images for unique_id 1021e07653b9660d014c758778ebd19e due to error: name 'xtal' is not defined
Details for unique_id 1022379ae32b33f912b010ea1823fff9:
Failed to get details for unique_id 1022379ae32b33f912b010ea1823fff9 due to error: cannot find the entry from 1022379ae32b33f912b010ea1823fff9
Failed to get spherical images for unique_id 1022379ae32b33f912b010ea1823fff9 due to error: name 'xtal' is not defined
Details for unique_id 1022d64f2007ce0d30f7ccda9bc7bb5d:
Failed to get details for unique_id 1022d64f2007ce0d30f7ccda9bc7bb5d due to error: cannot find the entry from 1022d64f2007ce0d30f7ccda9bc7bb5d
Failed to get spherical images for unique_id 1022d64f2007ce0d30f7ccda9bc7bb5d due to error: name 'xtal' is not defined
Details for unique_id 

KeyboardInterrupt: ignored

In [ ]:
import os
import pandas as pd
from pyxtal.db import database
from glob import glob
from pyxtal.msg import AtomTypeError

# 1) Compute the template descriptor

mech_db = database('mech.db')
template_code = 'DAHMUX'
template_xtal = mech_db.get_pyxtal(template_code)
template_sph = template_xtal.get_spherical_images()

results = []

count = 0

# Loop over all the Organic database files
for path_to_db in glob('dataset/Organic-*.db'):  # adding Organic-1,2,3,and 4
    organic_db = database(path_to_db)
    print(path_to_db, len(organic_db.codes))
    for code in organic_db.codes:
        #print(code)
        xtal = None
        try:
            xtal = organic_db.get_pyxtal(code)
        except:
            print("Reading Error in code", code)
        if xtal is not None:

            #print(xtal)
            try:
                sph = xtal.get_spherical_images()
                S = template_sph.get_similarity(sph, M=10, cutoff=0.95).mean()
                # Save the information about (unique_id, S)

                print("============={:4d} {:8s} {:6.3f}".format(count, code, S))
                results.append((code, S))
                count += 1
            except AtomTypeError:
                print("AtomType Error in code", code)

# Export the (unique_id, S) to the csv file.

df = pd.DataFrame(results, columns=['unique_id', 'similarity'])

df.to_csv('similarity.csv', index=False)

In [ ]:
import os
import pandas as pd
from pyxtal.db import database
from glob import glob
from pyxtal.msg import AtomTypeError

# 1) Compute the template descriptor
mech_db = database('mech.db')
template_code = 'DAHMUX'
template_xtal = mech_db.get_pyxtal(template_code)
template_sph = template_xtal.get_spherical_images()

results = []
count = 0

# Loop over all the Organic database files
for path_to_db in glob('dataset/Organic-*.db'):
    organic_db = database(path_to_db)
    print(f"Processing database: {path_to_db}, {len(organic_db.codes)} codes")

    for code in organic_db.codes:
        try:
            xtal = organic_db.get_pyxtal(code)
        except:
            print(f"Reading Error in code {code}")
            continue

        if xtal is not None:
            try:
                sph = xtal.get_spherical_images()
                S = template_sph.get_similarity(sph, M=10, cutoff=0.95).mean()

                # Save the information about (unique_id, S)
                print(f"============={count:4d} {code:8s} {S:6.3f}")
                results.append((code, S))
                count += 1
            except AtomTypeError:
                print(f"AtomType Error in code {code}")

# Print the current working directory and the total count of results
print(f"Current working directory: {os.getcwd()}")
print(f"Total results: {len(results)}")

# Export the (unique_id, S) to the csv file.
try:
    df = pd.DataFrame(results, columns=['unique_id', 'similarity'])
    df.to_csv('similarity.csv', index=False)
    print("Successfully wrote to similarity.csv")
except Exception as e:
    print(f"Error writing file: {e}")


Current working directory: /content
Total results: 0
Successfully wrote to similarity.csv


In [ ]:
import os
import pandas as pd
from pyxtal.db import database
from glob import glob
from pyxtal.msg import AtomTypeError

# 1) Compute the template descriptor
mech_db = database('mech.db')
template_code = 'DAHMUX'
template_xtal = mech_db.get_pyxtal(template_code)
template_sph = template_xtal.get_spherical_images()

results = []
count = 0

# List of Organic database files
organic_db_files = ['Organic-1.db', 'Organic-2.db', 'Organic-3.db', 'Organic-4.db']

# Loop over all the Organic database files
for path_to_db in organic_db_files:
    if not os.path.exists(path_to_db):
        print(f"Database file does not exist: {path_to_db}")
        continue

    organic_db = database(path_to_db)
    print(f"Processing database: {path_to_db}, {len(organic_db.codes)} codes")

    for code in organic_db.codes:
        try:
            xtal = organic_db.get_pyxtal(code)
        except:
            print(f"Reading Error in code {code}")
            continue

        if xtal is not None:
            try:
                sph = xtal.get_spherical_images()
                S = template_sph.get_similarity(sph, M=10, cutoff=0.95).mean()

                # Save the information about (unique_id, S)
                print(f"============={count:4d} {code:8s} {S:6.3f}")
                results.append((code, S))
                count += 1
            except AtomTypeError:
                print(f"AtomType Error in code {code}")

# Print the current working directory and the total count of results
print(f"Current working directory: {os.getcwd()}")
print(f"Total results: {len(results)}")

# Export the (unique_id, S) to the csv file.
try:
    df = pd.DataFrame(results, columns=['unique_id', 'similarity'])
    df.to_csv('similarity.csv', index=False)
    print("Successfully wrote to similarity.csv")
except Exception as e:
    print(f"Error writing file: {e}")


Processing database: Organic-1.db, 3836 codes
Reading Error in code AACFAZ10
AtomType Error in code ABAGOB
=============   0 ABAWEG    0.868
=============   1 ABAWOQ    0.905
=============   2 ABAWUW    0.645
=============   3 ABEDOA01  0.893
=============   4 ABEFOE    0.827
Reading Error in code ABIGOK


[08:01:05] UFFTYPER: Warning: hybridization set to SP3 for atom 7
[08:01:05] UFFTYPER: Warning: hybridization set to SP3 for atom 16


AtomType Error in code ABINUV
=============   5 ABIVIQ    0.460


In [ ]:
import os
import pandas as pd
from pyxtal.db import database
from glob import glob
from pyxtal.msg import AtomTypeError

# 1) Compute the template descriptor
mech_db = database('mech.db')
template_code = 'DAHMUX'
template_xtal = mech_db.get_pyxtal(template_code)
template_sph = template_xtal.get_spherical_images()

results = []
count = 0

# List of Organic database files
organic_db_files = ['Organic-1.db']

# Loop over all the Organic database files
for path_to_db in organic_db_files:
    if not os.path.exists(path_to_db):
        print(f"Database file does not exist: {path_to_db}")
        continue

    organic_db = database(path_to_db)
    print(f"Processing database: {path_to_db}, {len(organic_db.codes)} codes")

    for code in organic_db.codes:
        try:
            xtal = organic_db.get_pyxtal(code)
        except:
            print(f"Reading Error in code {code}")
            continue

        if xtal is not None:
            try:
                sph = xtal.get_spherical_images()
                S = template_sph.get_similarity(sph, M=10, cutoff=0.95).mean()

                # Save the information about (unique_id, S)
                print(f"============={count:4d} {code:8s} {S:6.3f}")
                results.append((code, S))
                count += 1
            except AtomTypeError:
                print(f"AtomType Error in code {code}")

# Print the current working directory and the total count of results
print(f"Current working directory: {os.getcwd()}")
print(f"Total results: {len(results)}")

# Export the (unique_id, S) to the csv file.
try:
    df = pd.DataFrame(results, columns=['unique_id', 'similarity'])
    df.to_csv('similarity.csv', index=False)
    print("Successfully wrote to similarity.csv")
except Exception as e:
    print(f"Error writing file: {e}")


Processing database: Organic-1.db, 3836 codes
Reading Error in code AACFAZ10
AtomType Error in code ABAGOB
=============   0 ABAWEG    0.868
=============   1 ABAWOQ    0.905
=============   2 ABAWUW    0.645
=============   3 ABEDOA01  0.893
=============   4 ABEFOE    0.827
Reading Error in code ABIGOK


[08:04:59] UFFTYPER: Warning: hybridization set to SP3 for atom 7
[08:04:59] UFFTYPER: Warning: hybridization set to SP3 for atom 16


AtomType Error in code ABINUV
=============   5 ABIVIQ    0.459
=============   6 ABOKUY    0.651
AtomType Error in code ABUREW
=============   7 ABUZED    0.706
=============   8 ACAGIU    0.818
=============   9 ACAGOA    0.390
AtomType Error in code ACAREB
=============  10 ACASOM    0.771
=============  11 ACAWEH    0.924
=============  12 ACEFOE    0.794
=============  13 ACEOXM    0.454
=============  14 ACEQOR    0.875
=============  15 ACETIM    0.588
=============  16 ACETYL    0.574
=============  17 ACICAR01  0.809
=============  18 ACIDEV    0.669
=============  19 ACIDOF    0.419
=============  20 ACIGUP    0.524
=============  21 ACIJOM    0.819
AtomType Error in code ACIREL
=============  22 ACIROU    0.810
=============  23 ACIRUA    0.792
AtomType Error in code ACIRUB
=============  24 ACISAH    0.765
=============  25 ACISAI    0.885
=============  26 ACIYAP    0.664
=============  27 ACIYIX    0.947
=============  28 ACOGIK    0.767
=============  29 ACOGOQ    0.727


[08:20:03] UFFTYPER: Unrecognized charge state for atom: 18
[08:20:03] UFFTYPER: Unrecognized charge state for atom: 19


=============  45 ADEFAT    0.865
=============  46 ADEKOK    0.758
=============  47 ADEQIK    0.656
=============  48 ADETAE    0.351
=============  49 ADEZIT    0.490
=============  50 ADEZOZ    0.893
Reading Error in code ADIJAY
=============  51 ADILEG    0.585
Reading Error in code ADINIL
=============  52 ADIPAC    0.890
=============  53 ADIPAM11  0.741
Reading Error in code ADIPOU
=============  54 ADMNTB03  0.575
=============  55 ADOHOS    0.848
AtomType Error in code ADOLEM
Reading Error in code ADUNIZ
=============  56 ADUQEW    0.924
=============  57 ADUQOG    0.924
=============  58 ADUQUM    0.816
=============  59 ADUWUS    0.879
=============  60 ADUXAZ    0.965


...

AtomType Error in code GOFHEP
=============2599 GOFQEY    0.571
=============2600 GOFRID    0.746
=============2601 GOFSIE    0.860
=============2602 GOFZAE    0.397
=============2603 GOGTUT    0.550
=============2604 GOGVEE    0.456
=============2605 GOGWUU    0.663
=============2606 GOGXEF    0.357
AtomType Error in code GOGXOP
Reading Error in code GOGZAG
Reading Error in code GOHHIV
Reading Error in code GOHJES
=============2607 GOHKEU    0.767
=============2608 GOHLAR    0.658
=============2609 GOHLOH    0.871
=============2610 GOJCIU    0.734
=============2611 GOJPUR    0.881
=============2612 GOJQUS    0.561
=============2613 GOJRAB    0.889
=============2614 GOJREF    0.652
Reading Error in code GOKNID
=============2615 GOKNOM    0.814
Reading Error in code GOKSAB
Reading Error in code GOKSIJ
=============2616 GOKZAJ    0.503
=============2617 GOLBUG    0.412
=============2618 GOLGOD01  0.748
=============2619 GOLJOH    0.586
=============2620 GOLJUN    0.931
Reading Error in c

[00:40:41] UFFTYPER: Warning: hybridization set to SP3 for atom 4
[00:40:41] UFFTYPER: Warning: hybridization set to SP3 for atom 13


=============2640 GOZMUD    0.806
=============2641 GOZNOY    0.920
Reading Error in code GUBBUD
=============2642 GUBKUM    0.619
=============2643 GUCLAU    0.907
=============2644 GUCNAU    0.802
Reading Error in code GUCWOQ01
=============2645 GUCZIN    0.599
=============2646 GUFBIS    0.887
=============2647 GUFLIC    0.550
AtomType Error in code GUFMAV
=============2648 GUFTOS    0.430
Reading Error in code GUGGIB
AtomType Error in code GUGHEX
AtomType Error in code GUGMIE
Reading Error in code GUGPOO
=============2649 GUHDIW    0.915
=============2650 GUHDUL    0.810
AtomType Error in code GUHJOL
=============2651 GUHKUP    0.807
=============2652 GUHMED    0.844
=============2653 GUHSAG    0.906
=============2654 GUJQEH    0.866
=============2655 GUJQIL    0.814
=============2656 GUKPEH    0.946
=============2657 GUKPIL    0.952
=============2658 GUKTAH    0.753
=============2659 GUKTOV    0.466
=============2660 GULDOG01  0.740
AtomType Error in code GUMQEL
=============2661 

[02:47:47] UFFTYPER: Unrecognized atom type: S_5+4 (1)
[02:47:47] UFFTYPER: Unrecognized atom type: S_5+4 (7)


AtomType Error in code HORQEK
AtomType Error in code HOSTAM
=============2989 HOSVUJ    0.767
=============2990 HOTAUR10  0.371
=============2991 HOTFIG    0.580
=============2992 HOTPIQ    0.582
=============2993 HOTXOE    0.889
Reading Error in code HOVFAC
=============2994 HOVKOS    0.538
=============2995 HOVLAF    0.875
=============2996 HOVYUO    0.470
Reading Error in code HOVZID
=============2997 HOWBAW    0.673
=============2998 HOWFUV    0.846
=============2999 HOWPAK    0.423
=============3000 HOXGAE    0.736
AtomType Error in code HOXGEI
=============3001 HOXGIM    0.768
=============3002 HOXMAJ    0.699
=============3003 HOXMEN    0.744
AtomType Error in code HOXNUF
=============3004 HOXOCD    0.855
AtomType Error in code HOYWEA
Current working directory: /content
Total results: 3005
Successfully wrote to similarity.csv


In [3]:
import os
import pandas as pd
from pyxtal.db import database
from glob import glob
from pyxtal.msg import AtomTypeError

# 1) Compute the template descriptor
mech_db = database('mech.db')
template_code = 'DAHMUX'
template_xtal = mech_db.get_pyxtal(template_code)
template_sph = template_xtal.get_spherical_images()

results = []
count = 0

# List of Organic database files
organic_db_files = ['Organic-2.db']

# Loop over all the Organic database files
for path_to_db in organic_db_files:
    if not os.path.exists(path_to_db):
        print(f"Database file does not exist: {path_to_db}")
        continue

    organic_db = database(path_to_db)
    print(f"Processing database: {path_to_db}, {len(organic_db.codes)} codes")

    for code in organic_db.codes:
        try:
            xtal = organic_db.get_pyxtal(code)
        except:
            print(f"Reading Error in code {code}")
            continue

        if xtal is not None:
            try:
                sph = xtal.get_spherical_images()
                S = template_sph.get_similarity(sph, M=10, cutoff=0.95).mean()

                # Save the information about (unique_id, S)
                print(f"============={count:4d} {code:8s} {S:6.3f}")
                results.append((code, S))
                count += 1
            except AtomTypeError:
                print(f"AtomType Error in code {code}")

# Print the current working directory and the total count of results
print(f"Current working directory: {os.getcwd()}")
print(f"Total results: {len(results)}")

# Export the (unique_id, S) to the csv file.
try:
    df = pd.DataFrame(results, columns=['unique_id', 'similarity'])
    df.to_csv('similarity.csv', index=False)
    print("Successfully wrote to similarity.csv")
except Exception as e:
    print(f"Error writing file: {e}")


Processing database: Organic-2.db, 3911 codes
Reading Error in code HOYWUQ
=============   0 HOYZED    0.819
=============   1 HOZREW    0.921
=============   2 HPYDZO10  0.491
=============   3 HSADAM    0.553
=============   4 HUBHUI    0.512
=============   5 HUBPAX    0.629
Reading Error in code HUBQAW
=============   6 HUBTON    0.707
AtomType Error in code HUBVAB
Reading Error in code HUBWEG
=============   7 HUCSAA    0.891
=============   8 HUCSIJ    0.463
=============   9 HUCYIO    0.625
=============  10 HUCZAH    0.691
=============  11 HUDFUI    0.901
AtomType Error in code HUDJOF
=============  12 HUFBAM    0.724
AtomType Error in code HUFCIW
=============  13 HUFJIC    0.663


[03:48:01] UFFTYPER: Warning: hybridization set to SP3 for atom 10
[03:48:01] UFFTYPER: Warning: hybridization set to SP3 for atom 13


AtomType Error in code HUFMED
=============  14 HUFQEF    0.959
=============  15 HUFSIL    0.901
=============  16 HUFTEH    0.478
=============  17 HUFTIM    0.894
AtomType Error in code HUFVEJ
=============  18 HUFWAJ    0.536
=============  19 HUFWIR    0.732
=============  20 HUFXEO    0.725
=============  21 HUFXIS    0.690
=============  22 HUFXOV    0.859
=============  23 HUGBUH    0.747
=============  24 HUGBUJ    0.374
=============  25 HUGCAQ    0.754
=============  26 HUGDIX    0.787
=============  27 HUGFIZ    0.770
=============  28 HUGMIH03  0.738
=============  29 HUGPEF    0.376
=============  30 HUGPUV    0.691
=============  31 HUGYAK    0.627
=============  32 HUGZOZ    0.610
=============  33 HUGZUF    0.464
=============  34 HUHCIX    0.648
Reading Error in code HUHKUR
=============  35 HUHWOW    0.677
AtomType Error in code HUHXAK
=============  36 HUJLED    0.873
=============  37 HUJLII    0.473
=============  38 HUJLIJ    0.761
=============  39 HUJLOO    0.5

/usr/local/lib/python3.10/dist-packages/pyxtal/descriptor.py:180: RuntimeWarning: invalid value encountered in double_scalars
  theta0 = np.arccos(r_vec[2]/r_mag)
/usr/local/lib/python3.10/dist-packages/pyxtal/descriptor.py:181: RuntimeWarning: invalid value encountered in double_scalars
  if abs((r_vec[2] / r_mag) - 1.0) < 10.**(-8.):
/usr/local/lib/python3.10/dist-packages/pyxtal/descriptor.py:183: RuntimeWarning: invalid value encountered in double_scalars
  elif abs((r_vec[2] / r_mag) + 1.0) < 10.**(-8.):


=============  48 HUKVUG      nan
=============  49 HUKYAQ01  0.953
=============  50 HULCUO    0.527
=============  51 HULHEB    0.483
=============  52 HULSEM    0.441
=============  53 HUMFIG    0.836
=============  54 HUMFOL    0.862
=============  55 HUMFUR    0.724
=============  56 HUMGAZ    0.682
Reading Error in code HUMJUU
=============  57 HUMLOR    0.831
=============  58 HUMRIR    0.875
=============  59 HUMRUC    0.889
=============  60 HUMSAK    0.602
=============  61 HUMTAN    0.947
=============  62 HUMTIV    0.622
=============  63 HUMTUF    0.539
=============  64 HUMVAP    0.917
=============  65 HUNHIJ    0.957
=============  66 HUNLOU    0.600
=============  67 HUNLUA    0.418
=============  68 HUNQEN    0.532
AtomType Error in code HUPBIG
=============  69 HUPZIC    0.520
=============  70 HUQNOW    0.626
=============  71 HURJEM    0.848
AtomType Error in code HURSIW
=============  72 HURSUI    0.811
=============  73 HUSQAP    0.907
Reading Error in code HUSRE

...

AtomType Error in code PUTXOT


[21:38:05] UFFTYPER: Warning: hybridization set to SP3 for atom 5


AtomType Error in code PUTXUZ
=============2832 PUVJUM    0.926
=============2833 PUVKAT    0.761
=============2834 PUVKIB    0.687
=============2835 PUVKOH    0.843
AtomType Error in code PUVMAV
=============2836 PUVNAW    0.688
AtomType Error in code PUWCIW
Reading Error in code PUWFEV
=============2837 PUWPUT    0.557
=============2838 PUWWAH    0.629
=============2839 PUXNAX    0.960
AtomType Error in code PUXPAB
=============2840 PUXXUE    0.474
=============2841 PUYCET    0.782
=============2842 PUYFUM    0.820
=============2843 PUYMEC    0.896
=============2844 PUYPII    0.529
AtomType Error in code PUYQOP
=============2845 PUYTOS    0.732
=============2846 PUYVEK    0.737
=============2847 PUZBAN    0.692
AtomType Error in code PUZRUY
=============2848 PUZWEM    0.679
AtomType Error in code PVVAWA01
=============2849 PYAZAC02  0.486
Reading Error in code PYEXDA
=============2850 PYRAZI    0.860
=============2851 PYRBTA01  0.712
=============2852 PYRBZR    0.913
=============285

[22:29:32] UFFTYPER: Warning: hybridization set to SP3 for atom 7
[22:29:32] UFFTYPER: Warning: hybridization set to SP3 for atom 10
[22:29:32] UFFTYPER: Warning: hybridization set to SP3 for atom 7
[22:29:32] UFFTYPER: Warning: hybridization set to SP3 for atom 10


AtomType Error in code QENRIN
Reading Error in code QEPBUK
=============2974 QEPNEG    0.810
=============2975 QEPTEM    0.784
=============2976 QEQJAY    0.685
=============2977 QEQKED    0.841
=============2978 QEQPUA    0.917
=============2979 QEQRUC    0.804
=============2980 QEQWAL    0.535
=============2981 QEQYIV    0.782
=============2982 QERMIJ    0.957
=============2983 QERNOT    0.873
=============2984 QERQAF    0.793
=============2985 QERTEN    0.671
=============2986 QERWAM01  0.670
=============2987 QERWEQ    0.681
AtomType Error in code QERWUF
=============2988 QERYAQ    0.688
=============2989 QERYES    0.937
=============2990 QERYEU    0.876
=============2991 QESKUU    0.762
=============2992 QESZUK    0.868
AtomType Error in code QETFAW
=============2993 QETFOK    0.872
Reading Error in code QETLUX
=============2994 QETVIU    0.811
=============2995 QETVOA    0.627
=============2996 QEVGAA    0.813
=============2997 QEVGED    0.823
=============2998 QEVQEQ    0.671
==